<a href="https://colab.research.google.com/github/jjssjj0607/analysis_of_petitions_about_animal/blob/main/03_02_%EC%96%B8%EC%96%B4%EB%B6%84%EC%84%9D_%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%8B%AC%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [ ]:
# konlpy 설치하기 ## https://data1000.tistory.com/33
!pip3 install jpype1==0.7.0
!pip3 install konlpy

     |████████████████████████████████| 2.7MB 7.0MB/s 
     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 92kB 6.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
# konlpy를 실행하기
import konlpy

In [ ]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [ ]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
## 문서 ID로 실제 파일 불러오기
# 실습을 위한 역대 대통령 취임사: 노무현, 이명박, 박근혜, 문재인
## https://drive.google.com/file/d/1LimuSrqR7ubVMH7ze_bc0nq4gk_DRiyt/view?usp=sharing
rawdata_downloaded = drive.CreateFile({'id': '1tea_wbrDE4iPcdYezdmjcxoDJdFoE9vn'})
rawdata_downloaded.GetContentFile('rawdata.csv')

In [ ]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
# rawdata_downloaded.GetContentFile('rawdata.csv') <<-- rawdata.csv 로 구글 드라이브에서 불러왔는데 정작 여기서는 rawdata.txt 로 읽어서 그렇습니다. 
원본데이터 = pd.read_csv('rawdata.csv')

In [ ]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,Unnamed: 0,number,progress,title,like,contents,answer,category,sday,eday,proponent,분류
0,1605,231701,답변완료,유기견 보호소 폐지를 막아주세요!!!,226252,대구 팔공산에 위치한 한나네 유기견 보호소를 도와주세요!!!\n대구 팔공산 한나네 ...,<정혜승 뉴미디어비서관> \n안녕하십니까. 뉴미디어비서관 정혜승입니다. 오늘 11시...,반려동물,2018-05-13,2018-06-12,facebook,유기동물
1,4308,580325,답변완료,이천에서 벌어진 동물수간사건에 대한 강력한 처벌과 동물학대에 대한 범국가적인 대책마...,217483,이천에서 생후 3개월 된 강아지를 길가던 행인이 수간과 함께 신체에 해를 가하는 사...,<정혜승 디지털소통센터장>\n이어 답변드릴 청원은 경기도 이천에서 벌어진 한 동물학...,반려동물,2019-05-18,2019-06-17,naver,동물학대
2,4295,579585,답변완료,**대 수의대에서 실험중인 퇴역 탐지견을 구조해주십시요,217249,안녕하십니까?\n(사)비글구조네트워크입니다\n\n오늘 2019년 4월 15일자 KB...,<정혜승 센터장> \n안녕하십니까. 청와대 디지털소통센터장 정혜승입니다. 오늘은 ‘...,반려동물,2019-04-12,2019-05-12,naver,동물학대
3,2038,272632,답변완료,개.고양이 식용종식 전동연(개를 가축에서 제외하라),214634,"대통령께 청원을 올리기에 앞서,\n\n소중한 생명으로 태어나 감금틀에서 극한 고통을...",<정혜승 디지털소통센터장> \n안녕하십니까. 디지털소통센터장 정혜승입니다. 오늘 1...,반려동물,2018-06-17,2018-07-17,facebook,도살
4,3491,454045,답변완료,도살장에서 망치로 머리를 맞던 개가 잠시 튀어나가 옆에있는 자신의 새끼에게 젖물리며...,214251,아래의 링크를 봐주세요\n사람과 똑같이 자신의 자식을 귀히여기고 사람과 똑같이 죽는...,<정혜승 디지털소통센터장> \n안녕하세요. 디지털소통센터장 정혜승입니다. 오늘 답변...,반려동물,2018-11-27,2018-12-27,naver,도살
...,...,...,...,...,...,...,...,...,...,...,...,...
175,4251,569334,청원종료,안산시 보건소 무분별한 쥐약 살포,2082,안산시 보건소 무분별한 쥐약 살포로인하여 죄 없는 강아지와 고양이가 죽어써요 \n관...,NaN,반려동물,2019-03-21,2019-04-20,naver,반려문화
176,2281,293611,청원종료,반려동물 분양금지 꼭 필요합니다.,2054,(가정견이라는 포장 속에 온라인 중고싸이트 거래에 택배배송까지 이루어집니다. 펫샵에...,NaN,반려동물,2018-07-02,2018-08-01,naver,개농장
177,1360,179421,청원종료,동물보호법 강화 부탁드립니다,2052,"""한 나라의 위대성과 그 도덕성은 동물들을 다루는 태도로 판단할 수 있다. 나는 나...",NaN,반려동물,2018-03-29,2018-04-28,facebook,반려문화
178,4397,584474,청원종료,반려동물 보유세관련 농림부 처벌 청원,2039,뉴스를 보니 유기동물을 관리하는 비용을 유기시키지 않고 잘 키우고 있는 반려인들에게...,NaN,반려동물,2020-01-13,2020-02-12,kakao,반려문화


# 형태소 분석

In [ ]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [ ]:
# "원본데이터"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터.iterrows(): # 원본데이터의 컬럼(열)을 불러와서 반복할 준비!
    대상텍스트 = row[5] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    분류 = row[11] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [ ]:
# "형태소" 변수의 저장 내용 확인
형태소

[('한', 'Verb'),
 ('유투버', 'Noun'),
 ('가', 'Josa'),
 ('고양이', 'Noun'),
 ('학대', 'Noun'),
 ('하며', 'Verb'),
 ("'", 'Punctuation'),
 ('칼빵', 'Noun'),
 ('하겠다', 'Verb'),
 ("'", 'Punctuation'),
 ('고', 'Noun'),
 ('말', 'Noun'),
 ('하고', 'Josa'),
 ('학대', 'Noun'),
 ('하는', 'Verb'),
 ('영상', 'Noun'),
 ('을', 'Josa'),
 ('올렸습니다', 'Verb'),
 ('\n\n', 'Foreign'),
 ('동물학대', 'Noun'),
 ('범', 'Noun'),
 ('은', 'Josa'),
 ('당당하게', 'Adjective'),
 ('자신', 'Noun'),
 ('이', 'Josa'),
 ('고양이', 'Noun'),
 ('를', 'Josa'),
 ('학대', 'Noun'),
 ('하는', 'Verb'),
 ('영상', 'Noun'),
 ('을', 'Josa'),
 ("'", 'Punctuation'),
 ('캐츠', 'Noun'),
 ('패치', 'Noun'),
 ('유튜브', 'Noun'),
 ("'", 'Punctuation'),
 ('에', 'Josa'),
 ('좌표', 'Noun'),
 ('를', 'Josa'),
 ('남겼고', 'Verb'),
 (',', 'Punctuation'),
 ('\n', 'Foreign'),
 ('영상', 'Noun'),
 ('안', 'Noun'),
 ('에서는', 'Josa'),
 ('동물학대', 'Noun'),
 ('범의', 'Noun'),
 ('장난스러운', 'Adjective'),
 ('말투', 'Noun'),
 ('와', 'Josa'),
 ('욕', 'Noun'),
 ('이', 'Josa'),
 ('담겨있고', 'Verb'),
 (',', 'Punctuation'),
 ('고양이', 'Noun'),
 ('를'

In [ ]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,한,Verb,동물학대,1
1,유투버,Noun,동물학대,1
2,가,Josa,동물학대,1
3,고양이,Noun,동물학대,1
4,학대,Noun,동물학대,1
...,...,...,...,...
113,시켜주십시오,Verb,동물학대,1
114,동물학,Noun,동물학대,1
115,대법,Noun,동물학대,1
116,강화,Noun,동물학대,1


In [ ]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[        0            1  type  count
 0      대구         Noun  유기동물      1
 1     팔공산         Noun  유기동물      1
 2       에         Josa  유기동물      1
 3      위치         Noun  유기동물      1
 4       한         Josa  유기동물      1
 ..    ...          ...   ...    ...
 226   !!!  Punctuation  유기동물      1
 227    공유         Noun  유기동물      1
 228    많이       Adverb  유기동물      1
 229  해주세요         Verb  유기동물      1
 230    !!  Punctuation  유기동물      1
 
 [231 rows x 4 columns],         0        1  type  count
 0      이천     Noun  동물학대      1
 1      에서     Josa  동물학대      1
 2      생후     Noun  동물학대      1
 3       3   Number  동물학대      1
 4      개월     Noun  동물학대      1
 ..    ...      ...   ...    ...
 287  \n\n  Foreign  동물학대      1
 288    서울     Noun  동물학대      1
 289  동물학대     Noun  동물학대      1
 290    방지     Noun  동물학대      1
 291    연합     Noun  동물학대      1
 
 [292 rows x 4 columns],           0            1  type  count
 0    안녕하십니까         Noun  동물학대      1
 1         ?  Punctuation  동물학

In [ ]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,대구,Noun,유기동물,1
1,팔공산,Noun,유기동물,1
2,에,Josa,유기동물,1
3,위치,Noun,유기동물,1
4,한,Josa,유기동물,1
...,...,...,...,...
113,시켜주십시오,Verb,동물학대,1
114,동물학,Noun,동물학대,1
115,대법,Noun,동물학대,1
116,강화,Noun,동물학대,1


In [ ]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,대구,Noun,유기동물,1
1,팔공산,Noun,유기동물,1
2,에,Josa,유기동물,1
3,위치,Noun,유기동물,1
4,한,Josa,유기동물,1
...,...,...,...,...
113,시켜주십시오,Verb,동물학대,1
114,동물학,Noun,동물학대,1
115,대법,Noun,동물학대,1
116,강화,Noun,동물학대,1


In [ ]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소  품사       분류  
\t   Foreign  동물학대      3
\n   Foreign  개농장     185
              도살      118
              동물학대    604
              반려문화    281
                     ... 
️⃣   Foreign  개농장      21
ｍ    Foreign  반려문화      2
🙏🏼   Foreign  개농장       4
🙏🙏🙏  Foreign  유기동물      1
🚫    Foreign  개농장       5
Name: 카운트, Length: 20625, dtype: int64

In [ ]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소 품사      분류       
\t  Foreign 동물학대    3
\n  Foreign 개농장   185
            도살    118
            동물학대  604
            반려문화  281
...               ...
️⃣  Foreign 개농장    21
ｍ   Foreign 반려문화    2
🙏🏼  Foreign 개농장     4
🙏🙏🙏 Foreign 유기동물    1
🚫   Foreign 개농장     5

[20625 rows x 1 columns]

In [ ]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')